# Working with MLflow

Introduction to the use of MLflow

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 06/01/2026   | Martin | Created   | Notebook to explore the use of MLflow | 

# Content

* [Introduction](#introduction)

# Introduction

Showcasing basic logging on mlflow

In [5]:
import pandas as pd
import pickle
import seaborn as sns
import mlflow
import matplotlib.pyplot as plt
import os

from dotenv import dotenv_values
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import root_mean_squared_error

In [6]:
config = dotenv_values("../.env")

os.environ["AWS_ACCESS_KEY_ID"] = config["MLFLOW_USER"]
os.environ["AWS_SECRET_ACCESS_KEY"] = config["MLFLOW_PASSWORD"]
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://127.0.0.1:9000"
os.environ["MLFLOW_S3_IGNORE_TLS"] = "true"

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [13]:
# Set the mlflow experiment
mlflow.set_experiment("nyc-taxi-experiment")

2026/01/06 20:53:55 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow/2', creation_time=1767704035465, experiment_id='2', last_update_time=1767704035465, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [ ]:
# Load model
def read_dataframe(filename):
  if filename.endswith('.csv'):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
  elif filename.endswith('.parquet'):
    df = pd.read_parquet(filename)

  df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
  df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

  df = df[(df.duration >= 1) & (df.duration <= 60)]

  categorical = ['PULocationID', 'DOLocationID']
  df[categorical] = df[categorical].astype(str)
  
  return df

df_train = read_dataframe('../data/raw/green_tripdata_2025-01.parquet')
df_val = read_dataframe('../data/raw/green_tripdata_2025-02.parquet')

In [9]:
# Preprocessing
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
with mlflow.start_run():
  mlflow.set_tag("developer", "martin") # Tags: For easier searching

  mlflow.log_param("train-data-path", "data/raw/green_tripdata_2025-01.parquet") # Data
  mlflow.log_param("valid-data-path", "data/raw/green_tripdata_2025-02.parquet")

  alpha = 0.01
  mlflow.log_param("alpha", alpha) # Params: Log model paramters
  lr = Lasso(alpha)
  lr.fit(X_train, y_train)


  y_pred = lr.predict(X_val)

  rmse = root_mean_squared_error(y_val, y_pred) # Metrics: Log model results
  mlflow.log_metric("rmse", rmse)

🏃 View run traveling-bird-593 at: http://127.0.0.1:5000/#/experiments/2/runs/4145f38d4f924777b17cd402df808bfb
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


In [ ]:
%load_ext watermark
%watermark